# utils

> util functions

In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from emb_opt.imports import *
from emb_opt.schemas import Batch, Query, Item

In [ ]:
#| export

def batch_list(inputs: List[Any],   # input list to be batched
               batch_size: int      # batch size
              ) -> List[List[Any]]: # batched output list
    '''
    batches the input list into chunks of size `batch_size`, with the last batch ragged
    
    if `batch_size=0`, returns list of all inputs
    '''
    if batch_size==0:
        output = [inputs]
    else:
        output = [inputs[i:i+batch_size] for i in range(0, len(inputs), batch_size)]
    return output

def unbatch_list(inputs: List[List[Any]] # input batched list
                ) -> List[Any]:          # flattened output list
    'flattens a batched list'
    return [item for sublist in inputs for item in sublist]

In [ ]:
inputs = list(range(10))
assert unbatch_list(batch_list(inputs, 3)) == inputs

In [ ]:
#| export

def build_batch_from_embeddings(embeddings: List[List[float]] # input embeddings
                               ) -> Batch:                    # output batch
    '''
    creates a `Batch` from a list of `embeddings`. Each embedding 
    is converted to a `Query` with a unique `collection_id`
    '''
    queries = []
    for i, embedding in enumerate(embeddings):
        query = Query.from_minimal(embedding=embedding)
        query.update_internal(collection_id=i)
        queries.append(query)
        
    batch = Batch(queries=queries) 
    return batch

In [ ]:
build_batch_from_embeddings([[0.1], [0.2]])

Batch(queries=[Query(item=None, embedding=[0.1], data={}, query_results=[], internal=InteralData(removed=False, removal_reason=None, parent_id=None, collection_id=0, iteration=None), id='query_191d47ea-5809-11ee-b05f-db94e348bdfb'), Query(item=None, embedding=[0.2], data={}, query_results=[], internal=InteralData(removed=False, removal_reason=None, parent_id=None, collection_id=1, iteration=None), id='query_191d47eb-5809-11ee-b05f-db94e348bdfb')])

In [ ]:
#| export

def build_batch_from_items(items: List[Item],      # input items
                           remap_collections=False # if collection ID should be remapped
                          ) -> Batch:              # output batch
    '''
    creates a `Batch` from a list of `Item` objects. Each `Item` 
    is converted to a `Query`. If `remap_collections=True`, each 
    `Query` is given a unique `collection_id`. Otherwise, each 
    `Query` retains the `collection_id` of the `Item` used to 
    create it
    '''
    queries = []
    for i, item in enumerate(items):
        query = Query.from_item(item)
        if remap_collections:
            query.update_internal(collection_id=i)
        queries.append(query)
    batch = Batch(queries=queries) 
    return batch

In [ ]:
build_batch_from_items([Item.from_minimal(embedding=[0.1])], remap_collections=True)

Batch(queries=[Query(item=None, embedding=[0.1], data={'_source_item_id': 'item_191d47ec-5809-11ee-b05f-db94e348bdfb'}, query_results=[], internal=InteralData(removed=False, removal_reason=None, parent_id=None, collection_id=0, iteration=None), id='query_191d47ed-5809-11ee-b05f-db94e348bdfb')])

In [ ]:
#| export

def whiten(scores: np.ndarray # vector shape (n,) of scores to whiten
          ) -> np.ndarray:    # vector shape (n,) whitened scores
    'Whitens vector of scores'
    mean = scores.mean()
    var = scores.var()
    
    return (scores - mean) / np.sqrt(var + 1e-8)

In [ ]:
#| export

def clip_grad(grad: np.ndarray,                  # grad vector
              max_norm: float,                   # max grad norm
              norm_type: Union[float, int, str]  # type of norm to use
             ):
    current_norm = np.linalg.norm(grad, ord=norm_type, axis=-1, keepdims=True)
    coef = np.minimum(max_norm / (current_norm + 1e-6), 1)
    grad = grad * coef
    return grad

In [ ]:
grad = np.array([1, 2, 3, 4, 5])
grads = np.stack([grad, grad])
assert (clip_grad(grad, 1., 2) == clip_grad(grads, 1., 2)[0]).all()

In [ ]:
#| export

def query_to_rl_inputs(query: Query) -> (np.ndarray, np.ndarray, np.ndarray):
    query_embedding = np.array(query.embedding)
    result_embeddings = np.array([i.embedding for i in query.valid_results()])
    scores = np.array([i.score for i in query.valid_results()])
    return query_embedding, result_embeddings, scores

In [ ]:
#| export 

def compute_rl_grad(query_embeddings: np.ndarray,                     # matrix of query embeddings
                    result_embeddings: np.ndarray,                    # matrix of result embeddings
                    result_scores: np.ndarray,                        # array of scores
                    distance_penalty: float=0,                        # distance penalty coefficient
                    max_norm: Optional[float]=None,                   # max gradient norm
                    norm_type: Optional[Union[float, int, str]]=2.0,  # type of norm to use
                    score_grad=False                                  # if loss should be score grad or loss grad
                   ):
    
    '''
    compute_rl_grad - uses reinforcement learning to estimate query gradients
    
    To compute the gradient with RL:
    1. compute advantages by whitening scores
        1. `advantage[i] = (scores[i] - scores.mean()) / scores.std()`
    2. compute advantage loss
        1. `advantage_loss[i] = advantage[i] * (query_embedding - result_embedding[i])**2`
    3. compute distance loss
        1. `distance_loss[i] = distance_penalty * (query_embedding - result_embedding[i])**2`
    4. sum loss terms
        1. `loss[i] = advantage_loss[i] + distance_loss[i]`
    5. compute the gradient
    
    This gives a closed for calculation of the gradient as:
    
    `grad[i] = 2 * (advantage[i] + distance_penalty) * (query_embedding - result_embedding[i])` 
    
    if `max_norm` is specified, the gradient will be clipped using `norm_type`
    
    if `score_grad=True`, the sign of the gradient is flipped. The standard sign is 
    designed for _minimizing_ the loss via gradient descent via `n_new = n_old - lr * grad`. 
    With the sign flipped, the gradient points directly in the direction of increasing score, 
    which is conceptually aligned with hill climbing, updating via `n_new = n_old + lr * grad`. 
    Use `score_grad=False` for anything using gradient descent.
    '''
    
    if query_embeddings.ndim==1:
        query_embeddings = query_embeddings[None]
        
    if result_embeddings.ndim==1:
        result_embeddings = result_embeddings[None]
        
    advantages = whiten(result_scores) + distance_penalty
    
    query_distances = query_embeddings[:,None,:] - result_embeddings[None,:,:]
    
    grad = (2 * advantages[None, :, None] * query_distances).mean(1)
    
    if max_norm is not None:
        assert norm_type is not None
        grad = clip_grad(grad, max_norm, norm_type)
        
    if score_grad:
        grad = -grad
    
    return np.squeeze(grad)